# ICRA 2022 Tutorial 3, Insufficient Data 2D (Range-only)

## Overview

- This example shows how non-Gaussian solutions (i.e. singular solutions) from having too few data given the desired system variables.
- In 2D on a flat plane, we imagine a robot traveling around in a rectangular trajectory making range measurements to beacons -- let's say WiFi power level is used as a proxy for range in this case.
- To illustrate the the underdetermined solution aspect, lets simplify the problem in that WiFi ranging (i.e. power level) is assumed as a pure unimodal (Gaussian) measurement.
- The non-Gaussian behavior will arise because we have more variables than measurement dimensions throughout the entire problem.
- The factor graph system will be built up in stages as the robot moves around the environment, which we will all pose/keyframe epochs.
- A new factor graph solve will be performed at each pose epoch, which produces the posterior marginal beliefs on each of the variables in the system.
- We assume the robot is traveling on a XY plane (viewing top down), starting at the origin along X and turning left along Y, then negative X, negative Y back to the origin. 
- Only four WiFi beacons are in the environment where the robot is moving.
- Measurements to the WiFj beacons can only be included with there if the signal is within range.
- Two of the beacon locations are known as prior information, while the other two beacons are at an unknown location and their location will be eastimated simultaneously to the robot pose location in the same factor graph system -- making this a minimalal simultaneous localization and mapping (SLAM) problem.
- To further simplify the tutorial, a range-only (i.e. distance-only) "odometry" measurement will be used between consecutive poses.  These "odometry" factors provide less information than conventional wheel or visual odometry constraints might provide.
- The tutorial is designed in such a way that the number of variable dimensions always exceed the measurement dimensions.
- After each pose epoch solution, we will look at the marginal belief estimates of all the variables in the system.
- This tutorial shows one of four mechanisms that can intoduce non-Gaussian behavior into a factor graph system, see other examples for other mechanisms.
- Optional, replace pure Gaussian measurement ranging models with ambiguous measurements shown in ICRA Tutorial 2.
- Optional, incorporate uncertain data association (i,e. multi-hypothesis) measurement models for unknown beacon associations similat to the technique used in ICRA Tutorial 4.
- Learn more from our peer-reviewed publications listed here: `CJLDocs/Literature`


### Signatures Used

`Point2`, `PriorPoint2`, `Point2Point2Range`, `MvNormal`, `Normal`.

## Loading The Data

The ground truth positions for vehicle positions GTp and landmark positions GTl can be loaded into memory directly with these values:

In [ ]:
GTp = Dict{Symbol, Vector{Float64}}()
GTp[:l100] = [0.0;0]
GTp[:l101] = [50.0;0]
GTp[:l102] = [100.0;0]
GTp[:l103] = [100.0;50.0]
GTp[:l104] = [100.0;100.0]
GTp[:l105] = [50.0;100.0]
GTp[:l106] = [0.0;100.0]
GTp[:l107] = [0.0;50.0]
GTp[:l108] = [0.0;0.0]

GTl = Dict{Symbol, Vector{Float64}}()
GTl[:l1] = [10.0;30]
GTl[:l2] = [30.0;-30]
GTl[:l3] = [80.0;40]
GTl[:l4] = [120.0;-50]

**Note,** 
- Using variable names `:l1, :l2, ...` or `:l100, :l101, ...` is simply for syntax benefit when visualizing with existing `RoMEPlotting` functions.
- Landmarks must be in range before range measurements can be made to them.

## Creating the Factor Graph with Position2



In [ ]:
using IncrementalInference, RoME

# TODO remove use of RoME here since new IIF.Position2 will be same as `RoME.Point2`

In [ ]:
# create the factor graph object
fg = initfg()

# first pose with no initial estimate
addVariable!(fg, :l100, Point2)

# add three landmarks
addVariable!(fg, :l1, Point2)
addVariable!(fg, :l2, Point2)
addVariable!(fg, :l3, Point2)

# and put priors on :l101 and :l102
addFactor!(fg, [:l1;], PriorPoint2(MvNormal(GTl[:l1], diagm(ones(2)))) )
addFactor!(fg, [:l2;], PriorPoint2(MvNormal(GTl[:l2], diagm(ones(2)))) )

The `PriorPoint2` is assumed to be a multivariate normal distribution of covariance `diagm(ones(2))`. Note the API `PriorPoint2(::T) where T <: SamplableBelief = PriorPoint2{T}` to accept distribution objects, discussed further in subsection [Various `SamplableBelief` Distribution types](https://juliarobotics.org/Caesar.jl/latest/concepts/dataassociation/#Various-SamplableBelief-Distribution-Types).

## Adding Range Measurements Between Variables

Next we connect the three range measurements from the vehicle location `:l0` to the three beacons, respectively – and consider that the range measurements are completely relative between the vehicle and beacon position estimates:

In [ ]:
# first range measurement
rhoZ1 = norm(GTl[:l1]-GTp[:l100])
ppr = Point2Point2Range( Normal(rhoZ1, 2) )
addFactor!(fg, [:l100;:l1], ppr)

# second range measurement
rhoZ2 = norm(GTl[:l2]-GTp[:l100])
ppr = Point2Point2Range( Normal(rhoZ2, 3.0) )
addFactor!(fg, [:l100; :l2], ppr)

# second range measurement
rhoZ3 = norm(GTl[:l3]-GTp[:l100])
ppr = Point2Point2Range( Normal(rhoZ3, 3.0) )
addFactor!(fg, [:l100; :l3], ppr)

The ranging measurement standard deviation of 2.0 or 3.0 is taken, assuming a Gaussian measurement assumption. Again, any distribution could have been used. The factor graph should look as follows:

In [ ]:
DFG.plotDFG(fg) # show the factor graph

## Inference and Visualizations

At this point we can call the solver start interpreting the first results:

In [ ]:
solveGraph!(fg)

The factor graph figure above showed the structure between variables and factors. In order to see the numerical values contained in the factor graph, a set of tools are provided by the RoMEPlotting and KernelDensityEstimatePlotting packages.

First look at the two landmark positions `:l1`, `:l2` at `(10.0,30)`, `(30.0,-30)` respectively.

In [ ]:
using RoMEPlotting

plotKDE(fg, [:l1;:l2], dims=[1;2])

Similarly, the belief estimate for the first vehicle position :l100 is bi-modal, due to the intersection of two range measurements:

In [ ]:
plotKDE(fg, :l100, dims=[1;2], levels=6)

An alternative plotting interface can also be used, that shows a histogram of desired elements instead:

In [ ]:
plotLandmarks(fg, from=1, to=101, contour=false, drawPoints=true)

Notice the ring of particles which represents the belief on the third beacon/landmark :l3, which was not constrained by a prior factor. Instead, the belief over the position of :l3 is being estimated simultaneous to estimating the vehicle position `:l100`.

## Implicit Growth and Decay of Modes (i.e. Hypotheses)

Next consider the vehicle moving a distance of 50 units–-and by design the direction of travel is not known–-to the next true position. The video above gives away the vehicle position with the cyan line, showing travel in the shape of a lower case 'e'. The following function handles (pseudo odometry) factors as range-only between positions and range-only measurement factors to beacons as the vehice travels.

In [ ]:
function vehicle_drives_to!(fgl::G, pos_sym::Symbol, GTp::Dict, GTl::Dict; measurelimit::R=150.0) where {G <: AbstractDFG, R <: Real}
  currvar = union(ls(fgl)...)
  prev_sym = Symbol("l$(maximum(Int[parse(Int,string(currvar[i])[2:end]) for i in 2:length(currvar)]))")
  if !(pos_sym in currvar)
    println("Adding variable vertex $pos_sym, not yet in fgl<:AbstractDFG.")
    addVariable!(fgl, pos_sym, Point2)
    @show rho = norm(GTp[prev_sym] - GTp[pos_sym])
    ppr = Point2Point2Range( Normal(rho, 3.0) )
    addFactor!(fgl, [prev_sym;pos_sym], ppr)
  else
    @warn "Variable node $pos_sym already in the factor graph."
  end
  beacons = keys(GTl)
  for ll in beacons
    rho = norm(GTl[ll] - GTp[pos_sym])
    # Check for feasible measurements:  vehicle within 150 units from the beacons/landmarks
    if rho < measurelimit
      ppr = Point2Point2Range( Normal(rho, 3.0) )
      if !(ll in currvar)
        println("Adding variable vertex $ll, not yet in fgl<:AbstractDFG.")
        addVariable!(fgl, ll, Point2)
      end
      addFactor!(fgl, [pos_sym;ll], ppr)
    end
  end
  nothing
end

After pasting (or running) this function in Julia, a new member definition vehicle_drives_to! can be used line any other function. Julia will handle the just-in-time compiling for the type specific function required and cach the static code for repeat executions.

**Note**, The exclamation mark at the end of the function name has no syntactic significance in Julia, since the full UTF8 character set is available for functions or variables. Instead, the exclamation serves as a Julia community convention to tell the caller that this function will modify the contents of at least some of the variables being passed into it – in this case the factor graph fg will be modified.

Now the actual driving event can be added to the factor graph:

In [ ]:
#drive to location :l101, then :l102
vehicle_drives_to!(fg, :l101, GTp, GTl)
vehicle_drives_to!(fg, :l102, GTp, GTl)

# see the graph
DFG.plotDFG(fg)

**Note**, the distance traveled could be any combination of accrued direction and speeds, however, a straight line Gaussian error model is used to keep the visual presentation of this example as simple as possible.

The marginal posterior estimates are found by repeating inference over the factor graph, followed drawing all vehicle locations as a contour map:

In [ ]:
# solve the graph
solveGraph!(fg)

# plot all vehicle locations
plotKDE(fg, [Symbol("l$(100+i)") for i in 0:2], dims=[1;2])

Notice how the vehicle positions have two hypotheses, one left to right and one diagonal right to bottom left – both are valid solutions!

In [ ]:
plotKDE(fg, [:l3;:l4], dims=[1;2], levels=4)

The two "free" beacons/landmarks :l3,:l4 still have several modes each, implying insufficient data to constrain either to a strong unimodal belief.

In [ ]:
vehicle_drives_to!(fg, :l103, GTp, GTl)
vehicle_drives_to!(fg, :l104, GTp, GTl)

tree = solveTree!(fg)

plotKDE(fg, [Symbol("l$(100+i)") for i in 0:4], dims=[1;2])

Moving up to position `:l104` still shows strong multiodality in the vehicle position estimates.

In [ ]:
vehicle_drives_to!(fg, :l105, GTp, GTl)
vehicle_drives_to!(fg, :l106, GTp, GTl)
solveGraph!(fg)


vehicle_drives_to!(fg, :l107, GTp, GTl)
solveGraph!(fg)


vehicle_drives_to!(fg, :l108, GTp, GTl)
solveGraph!(fg)


plotKDE(fg, [Symbol("l$(100+i)") for i in 2:8], dims=[1;2], levels=6)

Next we see a strong return to a single dominant mode in all vehicle position estimates, owing to the increased measurements to beacons/landmarks as well as more unimodal estimates in `:l3, :l4` beacon/landmark positions.

In [ ]:
vehicle_drives_to!(fg, :l109, GTp, GTl)
vehicle_drives_to!(fg, :l110, GTp, GTl)
solveGraph!(fg)


vehicle_drives_to!(fg, :l111, GTp, GTl)
vehicle_drives_to!(fg, :l112, GTp, GTl)
solveGraph!(fg)


pl = plotKDE(fg, [Symbol("l$(100+i)") for i in 7:12], dims=[1;2])

Several location belief estimates exhibit multimodality as the trajectory progresses (not shown), but collapses and finally collapses to a stable set of dominant position estimates.  Landmark estimates are also stable at one estimate:

In [ ]:
plotKDE(fg, [:l1;:l2;:l3;:l4], dims=[1;2], levels=4)

## A BIT MORE TO DO HERE

REDUCE THE EXISTING UNDERDETERMINED EXAMPLE TO JUST A RECTANGLE TRAJECTORY.
https://juliarobotics.org/Caesar.jl/latest/examples/basic_slamedonut/ 